# Install dependencies

In [ ]:
!pip -q install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.4 MB/s eta 0:00:00


In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00


In [ ]:
!pip -q install pillow transformers torch openpyxl

# Rough

In [ ]:
import os
os.chdir('/content/drive/MyDrive/invoice data extraction/')

In [ ]:
os.listdir()

['train1',
 'ghega-dataset',
 'structured_data',
 'train2',
 'train3',
 'main.ipynb']

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/invoice data extraction/train3.zip','r') as zipf:
  zipf.extractall('/content/drive/MyDrive/invoice data extraction/train3')

# Data Processing

In [ ]:
!sudo apt-get install tesseract-ocr
!pip -q install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,023 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import pytesseract
from PIL import Image
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, RagRetriever, RagTokenForGeneration
from datasets import Dataset

In [ ]:
# Define the paths
image_dir = 'train2/train/receipt'  # Directory with invoice images
output_dir = 'structured_data/invoice'  # Directory to save structured Excel files
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def count_files_in_directory(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

directory = 'your_directory_path'  # Replace with the actual path
file_count = count_files_in_directory(image_dir)
print(f"Number of files in directory: {file_count}")

Number of files in directory: 633


In [ ]:
def preprocess_image(image_path):
    #Load the image
    img = cv2.imread(image_path)
    #Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Apply thresholding (binarization)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    #Noise removal using morphological operations
    kernel = np.ones((1, 1), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
    #Dilation to connect broken text
    dilated = cv2.dilate(morph, kernel, iterations=1)
    #Deskew the image
    coords = np.column_stack(np.where(dilated > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = dilated.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskewed = cv2.warpAffine(dilated, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    #Resizing
    resized = cv2.resize(deskewed, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    #Sharpen the image
    kernel_sharp = np.array([[0, -1, 0],
                             [-1, 5, -1],
                             [0, -1, 0]])
    sharpened = cv2.filter2D(resized, -1, kernel_sharp)
    #Apply Gaussian blurring (optional)
    blurred = cv2.GaussianBlur(sharpened, (3, 3), 0)

    return blurred

In [ ]:
# Text extraction from images using pytesseract
def extract_text_from_images(image_dir):

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
    extracted_data = []
    count = 0
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        preprocessed_image = preprocess_image(image_path)
        preprocessed_pil_image = Image.fromarray(preprocessed_image)
        text = pytesseract.image_to_string(preprocessed_pil_image, lang='eng').strip()
        extracted_data.append({'image_file': image_file, 'text': text})
        count+=1
        if count==5:
          break
    return extracted_data

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 24.2 MB/s eta 0:00:00


In [ ]:
import easyocr

def extract_text_from_images_using_easyocr(image_dir):
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(['en'])  # Specify the language(s) you want to use
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
    extracted_data = []
    count = 0
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        img = Image.open(image_path)
        # Use EasyOCR to extract text from the image
        results = reader.readtext(image_path)
        # Combine the text from the results
        text = " ".join([result[1] for result in results])
        extracted_data.append({'image_file': image_file, 'text': text})
        count+=1
        if count==5:
          break
    return extracted_data


In [ ]:
# Test
extract_text_from_images(image_dir)

[{'image_file': '1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg',
  'text': '3.00\n45 90\n3.00\n\n5190\n060\n\n   \n\n5/26/2016 12-53-10 PM\n\nTHANK You!\n\neee'},
 {'image_file': '1000-receipt_jpg.rf.b17cbc08490974652c2d9086e1e50dca.jpg',
  'text': ''},
 {'image_file': '1001-receipt_jpg.rf.109e88972a60051c211342c57973d540.jpg',
  'text': 'ol\n\nGaRAnNrP Ke\n\npocesvelt heie\nqaeve 100 Sharky\na v8:43\n\nyee\n\nas2s/\n\n   \n \n\n048°\nSTEPHEN F\n\ncoffee\nCotten parses\nOne Top SNTlON\nBr 1s gcon-cneese Pe\ncarter\n\neae'},
 {'image_file': '1001-receipt_jpg.rf.324f4e0a4f5c9ef2a0a2a6857e6c7b51.jpg',
  'text': 'i.\n\nGrRanpPt aK\n\nevelt Feld\n\n \n\n \n\n \n\n \n\n \n\nRoost\naso THEI Mera\nOs R sce: 4 fares 04/25/19\ncoffee. 2.98\nCotten Parmesan 17.38\nCheon sir torn 23.35\nPrime ron cheese Surge 13\ntotfee 2\na totals, 3168\nee\nEen. se\na your" ©\ngravutty Wot SRT\naCe AE\npes. pean 2 PISA to serve YU"\nPleasiining with YS\n\n————'},
 {'image_file': '1001-receipt_jpg.rf.4

In [ ]:
extract_text_from_images_using_easyocr(image_dir)

[{'image_file': '1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg',
  'text': 'GAEEN Fleld T CaciF COas WVN L"no Daach unnnu ano 59(-0906 stulion SuynI Jc!e Dnr 0t os 64974 lueet & Table 45-40 Wacn Coka SuD IQAL: Tox ~ToTal : 456.50 5/26/2016 12.:i0 Fh [HANK\' You ! Ued'},
 {'image_file': '1000-receipt_jpg.rf.b17cbc08490974652c2d9086e1e50dca.jpg',
  'text': "GHEEN FlelD, MuY PacIF 4manu Kon Booch 701406 (562) 597 Au Stel 0uicu DVni Gueglu Taba unch Coln SuO ioiai 556.58  ToTAL: I2 $J i0 Fh 5/26/2016 ihahk You! COAs[ JJu', 64424"},
 {'image_file': '1001-receipt_jpg.rf.109e88972a60051c211342c57973d540.jpg',
  'text': 'GRAND JESTE: 101 #Party TABLE 2.95 17.95 25.92 Cof fee 13195 Chicken Pri0e Top Burper Tha eacon 42r Cott Subs 8" 04/25 uderd Not Inc Grateated  13.83 T07 Ft VQu t0 Seriel tor Jinino TThank Tieid Rooseve!E 04/25/15 40:43 0480 Svrc: \'stepher Patuesun sirlain Cngeie Terala a Htata |F Sub TOTA 47;42 Gralu: Suqoar pleasure Hith been You'},
 {'image_file': '1001-receipt_

In [ ]:

import re

def preprocess_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(' +', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z0-9., ]', '', text)
    text = text.lower()

    return text

# GPT 2

In [ ]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# Initialize GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_table_from_text(extracted_text):
    # Prepare the prompt
    prompt = "Give title and construct table with the following data as JSON:\n" + extracted_text

    # Tokenize and generate text using GPT-2
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=1024, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

    print("prompt given..")
    # Decode the output and extract the JSON
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("text generated by gpt..")
    try:
        print("structuring text..")
        json_start = generated_text.index('{')
        json_end = generated_text.rindex('}') + 1
        json_data = generated_text[json_start:json_end]
        table_data = json.loads(json_data)  # Convert JSON text to Python dict
    except (ValueError, json.JSONDecodeError):
        print("Error: Generated text does not contain valid JSON.")
        return None

    return table_data


In [ ]:
def save_json_as_csv(table_data, output_file):
    if table_data:
        df = pd.DataFrame(table_data)  # Convert JSON dict to pandas DataFrame
        df.to_csv(output_file, index=False)  # Save DataFrame to CSV
        print(f"Saved structured data to {output_file}")

In [1]:

print("Extracting text from images...")
extracted_data = [{'image_file': '1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg',
  'text': 'GAEEN Fleld T CaciF COas WVN L"no Daach unnnu ano 59(-0906 stulion SuynI Jc!e Dnr 0t os 64974 lueet & Table 45-40 Wacn Coka SuD IQAL: Tox ~ToTal : 456.50 5/26/2016 12.:i0 Fh [HANK\' You ! Ued'}]# extract_text_from_images(image_dir)

print("Processing extracted data with GPT-2 to generate tables...")
for data in extracted_data:
  extracted_text = data['text']
  image_file = data['image_file']

  # Generate structured table as JSON
  table_data = generate_table_from_text(extracted_text)
  print(table_data)
  break

  #if table_data:
  #  output_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}.csv")
  #  save_json_as_csv(table_data, output_path)

'\nprint("Extracting text from images...")\nextracted_data = [{\'image_file\': \'1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg\',\n  \'text\': \'GAEEN Fleld T CaciF COas WVN L"no Daach unnnu ano 59(-0906 stulion SuynI Jc!e Dnr 0t os 64974 lueet & Table 45-40 Wacn Coka SuD IQAL: Tox ~ToTal : 456.50 5/26/2016 12.:i0 Fh [HANK\' You ! Ued\'}]# extract_text_from_images(image_dir)\n\nprint("Processing extracted data with GPT-2 to generate tables...")\nfor data in extracted_data:\n  extracted_text = data[\'text\']\n  image_file = data[\'image_file\']\n\n  # Generate structured table as JSON\n  table_data = generate_table_from_text(extracted_text)\n  print(table_data)\n  break\n'

# GPT 4 pipeline

In [ ]:
from transformers import pipeline

In [ ]:
gpt_pipeline = pipeline("text-generation", model="nomic-ai/gpt4all-j")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
input_text = "GANTD LTX C3FE RooseveIL Fleld 0480 TABLE 101 #Party STEPHEN SvrCk : \'16:43 04/25/15 Coffee 13.95 Chicken Parmesan Prime rioin 25.95 The Bacon Chelain Burger 13.95 Cotfee Subi Total: 63.75 Tax: Sub Total {69.25 04/25 17;47 TOTAL 6 18 Gratuity Not-Included Suguested Gratuity 203 13.85 18* 12.47 451 been pleasure tO serve You Thank You for dining with Us'}"
prompt = f"Give title to the table and construct a table with the following data, and return the table as JSON:\n\n{input_text}"

response = gpt_pipeline(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']

response

In [ ]:
def fine_tune_gpt4(extracted_data):
    # Initialize GPT-4 model pipeline (using text generation model from Hugging Face)
    gpt_pipeline = pipeline("text-generation", model="nomic-ai/gpt4all-j")

    structured_data = []

    for data in extracted_data:
        input_text = data['text']
        # Formulate the prompt for table generation
        prompt = f"Give title to the table and construct a table with the following data, and return the table as JSON:\n\n{input_text}"

        # Generate table as JSON using the model
        response = gpt_pipeline(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']

        structured_data.append({'image_file': data['image_file'], 'structured_json': response})

    return structured_data

In [ ]:
def json_to_csv_and_save(structured_data):
    for data in structured_data:
        json_text = data['structured_json']

        try:
            # Convert the JSON text into a DataFrame
            df = pd.read_json(json_text)

            # Save DataFrame to CSV
            output_path = os.path.join(output_dir, f"{os.path.splitext(data['image_file'])[0]}.csv")
            df.to_csv(output_path, index=False)
            print(f"Saved {output_path}")
        except ValueError:
            print(f"Failed to process JSON for {data['image_file']}")


In [ ]:
print("Extracting text from images...")
extracted_data = extract_text_from_images(image_dir)

Extracting text from images...


In [ ]:
extracted_data

[{'image_file': '1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg',
  'text': '3.00\n45 90\n3.00\n\n5190\n060\n\n   \n\n5/26/2016 12-53-10 PM\n\nTHANK You!\n\neee'},
 {'image_file': '1000-receipt_jpg.rf.b17cbc08490974652c2d9086e1e50dca.jpg',
  'text': ''},
 {'image_file': '1001-receipt_jpg.rf.109e88972a60051c211342c57973d540.jpg',
  'text': 'ol\n\nGaRAnNrP Ke\n\npocesvelt heie\nqaeve 100 Sharky\na v8:43\n\nyee\n\nas2s/\n\n   \n \n\n048°\nSTEPHEN F\n\ncoffee\nCotten parses\nOne Top SNTlON\nBr 1s gcon-cneese Pe\ncarter\n\neae'},
 {'image_file': '1001-receipt_jpg.rf.324f4e0a4f5c9ef2a0a2a6857e6c7b51.jpg',
  'text': 'i.\n\nGrRanpPt aK\n\nevelt Feld\n\n \n\n \n\n \n\n \n\n \n\nRoost\naso THEI Mera\nOs R sce: 4 fares 04/25/19\ncoffee. 2.98\nCotten Parmesan 17.38\nCheon sir torn 23.35\nPrime ron cheese Surge 13\ntotfee 2\na totals, 3168\nee\nEen. se\na your" ©\ngravutty Wot SRT\naCe AE\npes. pean 2 PISA to serve YU"\nPleasiining with YS\n\n————'},
 {'image_file': '1001-receipt_jpg.rf.4

In [ ]:
print("Generating structured table using GPT-4...")
structured_data = fine_tune_gpt4(extracted_data)

Generating structured table using GPT-4...


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("Saving structured data to CSV files...")
json_to_csv_and_save(structured_data)
print("Process completed!")

# GPT4o

In [ ]:
!pip -q install gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00


In [ ]:
from gpt4all import GPT4All

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="nomic-ai/gpt4all-j")

In [ ]:
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM
with model.chat_session():
    print(model.generate("How can I run LLMs efficiently on my laptop?", max_tokens=1024))

Downloading:  30%|██▉       | 1.39G/4.66G [01:13<01:28, 37.1MiB/s]

In [2]:
gpt = GPT4All(model_name = "ggml-gpt4all-j-v1.3-groovy.bin", model_path = "/content/drive/MyDrive/invoice data extraction/ggml-gpt4all-j-v1.3-groovy.bin")

In [3]:
response = gpt4all_model.chat('hello')

In [ ]:
def fine_tune_gpt4all(extracted_data):
    structured_data = []

    for data in extracted_data:
        # Define the prompt for GPT-4All
        prompt = f"Give title to the table and construct table with the following data: {data}. Return the table as JSON."

        # Generate the table in JSON format
        response = gpt4all_model.chat(prompt)
        generated_json = response['choices'][0]['message']['content']

        structured_data.append({
            'image_file': data['image_file'],
            'json_table': generated_json
        })

    return structured_data

In [ ]:
# Step 3: Convert the structured JSON to CSV and save
def save_to_csv(structured_data):
    for data in structured_data:
        json_table = data['json_table']
        image_file = data['image_file']

        # Convert JSON table to a pandas DataFrame
        df = pd.read_json(json_table)

        # Save DataFrame to CSV
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}.csv")
        df.to_csv(output_path, index=False)

In [ ]:
print("Extracting text from images...")
extracted_data = extract_text_from_images(image_dir)

Extracting text from images...


In [ ]:
extracted_data

[{'image_file': '1000-receipt_jpg.rf.57cc25581e55df4c4e839890fe81ee00.jpg',
  'text': '3.00\n45 90\n3.00\n\n5190\n060\n\n   \n\n5/26/2016 12-53-10 PM\n\nTHANK You!\n\neee'},
 {'image_file': '1000-receipt_jpg.rf.b17cbc08490974652c2d9086e1e50dca.jpg',
  'text': ''},
 {'image_file': '1001-receipt_jpg.rf.109e88972a60051c211342c57973d540.jpg',
  'text': 'ol\n\nGaRAnNrP Ke\n\npocesvelt heie\nqaeve 100 Sharky\na v8:43\n\nyee\n\nas2s/\n\n   \n \n\n048°\nSTEPHEN F\n\ncoffee\nCotten parses\nOne Top SNTlON\nBr 1s gcon-cneese Pe\ncarter\n\neae'},
 {'image_file': '1001-receipt_jpg.rf.324f4e0a4f5c9ef2a0a2a6857e6c7b51.jpg',
  'text': 'i.\n\nGrRanpPt aK\n\nevelt Feld\n\n \n\n \n\n \n\n \n\n \n\nRoost\naso THEI Mera\nOs R sce: 4 fares 04/25/19\ncoffee. 2.98\nCotten Parmesan 17.38\nCheon sir torn 23.35\nPrime ron cheese Surge 13\ntotfee 2\na totals, 3168\nee\nEen. se\na your" ©\ngravutty Wot SRT\naCe AE\npes. pean 2 PISA to serve YU"\nPleasiining with YS\n\n————'},
 {'image_file': '1001-receipt_jpg.rf.4

In [4]:
print("Generating structured tables using GPT-4All...")
structured_data = fine_tune_gpt4all(extracted_data)

Generating structured tables using GPT-4All...


In [ ]:
print("Saving structured tables to CSV files...")
save_to_csv(structured_data)
print("Process completed!")